<a href="https://colab.research.google.com/github/Nikhild2710/Movie-recommendations/blob/main/Nikhil_Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎬 Movie Recommendation System (Content + User-Based)
Built by Nikhil Dulipala –

**Techniques:**
- Popularity-based recommendations
- Content-based filtering using TF-IDF and cosine similarity
- User-user collaborative filtering using cosine similarity

**Datasets Required:**
- TMDB: `tmdb_5000_movies.csv`, `tmdb_5000_credits.csv`
- MovieLens: `ratings.csv`, `movies.csv`

In [ ]:
# Pin NumPy 1.x and install scikit-surprise, then force a restart
!pip -q install numpy==1.26.4
!pip -q install scikit-surprise==1.1.3

import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 55.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
# Always run this first in a new session
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd  # <-- You were missing this
import numpy as np

# Point to your Drive-based data directory
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/.kaggle"
DATA_DIR = "/content/drive/MyDrive/movie-recs-data"

# Load from Drive (no re-download)
tmdb_movies  = pd.read_csv(f"{DATA_DIR}/tmdb_5000_movies.csv")
tmdb_credits = pd.read_csv(f"{DATA_DIR}/tmdb_5000_credits.csv")
ml_movies    = pd.read_csv(f"{DATA_DIR}/movie.csv")
ml_ratings   = pd.read_csv(f"{DATA_DIR}/rating.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



movies = tmdb_movies.merge(tmdb_credits, left_on='id', right_on='movie_id')
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [9]:
movies.head(2).to_csv("sample_movies.csv", index=False)

In [10]:
from google.colab import files
files.download("sample_movies.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 📊 Load and Explore TMDB Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

movies = movies.merge(credits, left_on='id', right_on='movie_id')
movies.head(2)

In [7]:
movies.head(2).to_csv("sample_movies.csv", index=False)

## ⭐ Popularity-Based Recommender

In [4]:
# Use 'title_x' instead of 'title'
popularity_df = movies[['title_x', 'vote_count', 'vote_average']]
popularity_df = popularity_df.sort_values(by='vote_count', ascending=False)

# Optionally rename column to clean it
popularity_df = popularity_df.rename(columns={'title_x': 'title'})

# Display top 10 popular movies
popularity_df.head(10)

,title,vote_count,vote_average
96,Inception,13752,8.1
65,The Dark Knight,12002,8.2
0,Avatar,11800,7.2
16,The Avengers,11776,7.4
788,Deadpool,10995,7.4
95,Interstellar,10867,8.1
287,Django Unchained,10099,7.8
94,Guardians of the Galaxy,9742,7.9
426,The Hunger Games,9455,6.9
127,Mad Max: Fury Road,9427,7.2


In [5]:
movies.columns.tolist()


['budget',
 'genres',
 'homepage',
 'id',
 'keywords',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title_x',
 'vote_average',
 'vote_count',
 'movie_id',
 'title_y',
 'cast',
 'crew']

## 🧠 Content-Based Filtering with TF-IDF + Cosine Similarity

In [6]:
import ast

def safe_eval(x):
    try:
        return ast.literal_eval(x)
    except:
        return []

def names(list_of_dicts, key='name', top=None):
    vals = [d.get(key,'') for d in list_of_dicts if isinstance(d, dict)]
    return vals[:top] if top else vals

# Parse JSON-like columns
movies['genres_list']   = movies['genres'].apply(safe_eval)
movies['keywords_list'] = movies['keywords'].apply(safe_eval)
movies['cast_list']     = movies['cast'].apply(safe_eval)
movies['crew_list']     = movies['crew'].apply(safe_eval)

# Extract director name from crew
def get_director(crew):
    for person in crew:
        if isinstance(person, dict) and person.get('job') == 'Director':
            return person.get('name','')
    return ''

movies['director'] = movies['crew_list'].apply(get_director)

# Flatten to simple strings (limit cast to top 3 for signal/noise)
movies['genres_txt']   = movies['genres_list'].apply(lambda L: " ".join(names(L)))
movies['keywords_txt'] = movies['keywords_list'].apply(lambda L: " ".join(names(L)))
movies['cast_txt']     = movies['cast_list'].apply(lambda L: " ".join(names(L, top=3)))
movies['tagline']      = movies['tagline'].fillna('')
movies['director']     = movies['director'].fillna('')


In [12]:
movies['combined_features'] = (
    movies['keywords_txt'] + " " +
    movies['genres_txt']   + " " +
    movies['tagline']      + " " +
    movies['cast_txt']     + " " +
    movies['director']
).str.replace(r'\s+', ' ', regex=True).str.strip()


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['combined_features'].fillna(''))
cosine_sim = cosine_similarity(tfidf_matrix)


In [14]:
movies = movies.rename(columns={'title_x':'title'})
# If you like, drop title_y now: movies = movies.drop(columns=['title_y'], errors='ignore')

# Case-insensitive index (drop duplicates keeps first occurrence)
indices = pd.Series(movies.index, index=movies['title'].str.lower()).drop_duplicates()


In [15]:
def get_recommendations(title, cosine_sim=cosine_sim, topn=10):
    key = title.strip().lower()
    if key not in indices:
        raise KeyError(f"Title not found: {title}. Try exact TMDB title or check casing.")
    idx = indices[key]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:topn+1]
    movie_indices = [i for i,_ in sim_scores]
    return movies.loc[movie_indices, ['title','vote_average','vote_count']]

# Example
get_recommendations("The Dark Knight Rises")


,title,vote_average,vote_count
65,The Dark Knight,8.2,12002
119,Batman Begins,7.5,7359
428,Batman Returns,6.6,1673
210,Batman & Robin,4.2,1418
1359,Batman,7.0,2096
303,Catwoman,4.2,808
1196,The Prestige,8.0,4391
72,Suicide Squad,5.9,7458
1010,Panic Room,6.5,1267
299,Batman Forever,5.2,1498


## 👥 User-User Collaborative Filtering (Cosine Similarity)

In [2]:
import pandas as pd, numpy as np
from pathlib import Path

# Use already-loaded data if it exists; else load from Drive
if 'ml_ratings' in globals():
    ratings = ml_ratings[['userId','movieId','rating','timestamp']].copy()
else:
    ratings = pd.read_csv(f"{DATA_DIR}/rating.csv", usecols=['userId','movieId','rating','timestamp'])
if 'ml_movies' in globals():
    ml_movies_df = ml_movies[['movieId','title']].copy()
else:
    ml_movies_df = pd.read_csv(f"{DATA_DIR}/movie.csv", usecols=['movieId','title'])

ratings.head(3), movies.head(3)


(   userId  movieId  rating            timestamp
 0       1        2     3.5  2005-04-02 23:53:47
 1       1       29     3.5  2005-04-02 23:31:16
 2       1       32     3.5  2005-04-02 23:33:39,
    movieId                    title
 0        1         Toy Story (1995)
 1        2           Jumanji (1995)
 2        3  Grumpier Old Men (1995))

In [3]:
# Take ~1M random ratings for speed/stability
ratings_s = ratings.sample(n=min(1_000_000, len(ratings)), random_state=42).copy()

# Keep active users & popular movies (tune thresholds if RAM is tight)
u_counts = ratings_s['userId'].value_counts()
i_counts = ratings_s['movieId'].value_counts()

active_users   = u_counts[u_counts >= 100].index
popular_movies = i_counts[i_counts >= 500].index

ratings_s = ratings_s[ratings_s['userId'].isin(active_users) & ratings_s['movieId'].isin(popular_movies)].copy()
ratings_s.shape


(4861, 4)

In [4]:
def per_user_split(df, test_frac=0.2, seed=42):
    rng = np.random.default_rng(seed)
    mask = np.zeros(len(df), dtype=bool)
    for uid, idx in df.groupby('userId').indices.items():
        idx = np.array(idx)
        if len(idx) <= 1:
            continue
        take = max(1, int(round(test_frac * len(idx))))
        pick = rng.choice(idx, size=take, replace=False)
        mask[pick] = True
    test  = df[mask].copy()
    train = df[~mask].copy()
    return train, test

train, test = per_user_split(ratings_s, test_frac=0.2, seed=42)
len(train), len(test)

(3883, 978)

In [8]:
from scipy.sparse import csr_matrix

# Compact 0..N-1 codes
u_cat  = pd.Categorical(train['userId'])
i_cat  = pd.Categorical(train['movieId'])
u_codes = u_cat.codes
i_codes = i_cat.codes
n_u, n_i = len(u_cat.categories), len(i_cat.categories)

# User mean ratings (for centering)
user_means = train.groupby('userId')['rating'].mean().astype(float)

# Center ratings: r' = r - mean(user)
r_centered = train['rating'].values - user_means.loc[train['userId']].values

# Sparse matrix (users as rows, items as cols)
R = csr_matrix(
    (r_centered.astype(np.float32), (u_codes, i_codes)),
    shape=(n_u, n_i)
)

# ID↔code maps
code_to_user  = pd.Series(u_cat.categories)                           # row → userId
code_to_item  = pd.Series(i_cat.categories)                           # col → movieId
user_to_code  = pd.Series(range(n_u), index=u_cat.categories)         # userId → row
item_to_code  = pd.Series(range(n_i), index=i_cat.categories)         # movieId → col

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

GLOBAL_MEAN = float(train['rating'].mean())

def predict_user_user(user_id, movie_id, K=50):
    # Fallbacks if unseen in train
    if user_id not in user_to_code.index or movie_id not in item_to_code.index:
        return GLOBAL_MEAN

    u = int(user_to_code[user_id])
    j = int(item_to_code[movie_id])

    # similarities: user u vs ALL users (on centered ratings)
    sims = cosine_similarity(R[u], R).ravel()
    sims[u] = 0.0  # exclude self

    # who rated item j? (non-zero centered rating means they rated it)
    col_j = R.getcol(j).toarray().ravel()
    rated_mask = col_j != 0

    sims_item = sims * rated_mask
    if sims_item.sum() == 0:
        return float(user_means.get(user_id, GLOBAL_MEAN))

    # Top-K neighbors among raters of item j
    k_actual = int(min(K, (sims_item > 0).sum())) or 1
    top_idx = np.argpartition(-sims_item, k_actual-1)[:k_actual]
    top_idx = top_idx[np.argsort(-sims_item[top_idx])]

    nbr_sims = sims_item[top_idx]            # similarities (weights)
    nbr_ctr  = col_j[top_idx]                # neighbors' centered ratings for item j

    num = float(np.dot(nbr_sims, nbr_ctr))
    den = float(np.abs(nbr_sims).sum())
    if den == 0:
        return float(user_means.get(user_id, GLOBAL_MEAN))

    # add back user's mean to get predicted rating
    return float(user_means.get(user_id, GLOBAL_MEAN) + num/den)


In [10]:
from sklearn.metrics import mean_squared_error
from math import sqrt

TEST_CAP = 50_000  # keep evaluation fast
test_eval = test.sample(n=min(TEST_CAP, len(test)), random_state=42)

preds, acts = [], []
for _, row in test_eval.iterrows():
    preds.append(predict_user_user(int(row.userId), int(row.movieId), K=50))
    acts.append(float(row.rating))

rmse = sqrt(mean_squared_error(acts, preds))
print(f"User–User Cosine RMSE (n={len(test_eval)}): {rmse:.4f}")


User–User Cosine RMSE (n=978): 1.0788


In [12]:

from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split as sp_split
from surprise import accuracy

reader = Reader(rating_scale=(0.5, 5.0))
data   = Dataset.load_from_df(ratings_s[['userId','movieId','rating']], reader)
trainset, testset = sp_split(data, test_size=0.2, random_state=42)

svd = SVD(n_factors=100, random_state=42)
svd.fit(trainset)
preds = svd.test(testset)
svd_rmse = accuracy.rmse(preds, verbose=False)   # <-- capture it
print(f"SVD RMSE: {svd_rmse:.4f}")

SVD RMSE: 0.8831


In [13]:
print("Results (same split)")
print(f"User–User Cosine RMSE: {rmse:.4f}")   # from your CF block
print(f"SVD RMSE:             {svd_rmse:.4f}")

Results (same split)
User–User Cosine RMSE: 1.0788
SVD RMSE:             0.8831
